In [82]:
import glob
import sys,os
import json
import pprint
import time
import re
import datetime
import pickle
import string
import gc
import warnings
import yaml
warnings.filterwarnings("ignore")
sys.path.append(os.pardir)
sys.path.append('../..')
sys.path.append('../../..')

import numpy as np
import pandas as pd
import pandas_profiling as pdp
import matplotlib.pyplot as plt
import japanize_matplotlib # 日本語対応
import seaborn as sns
# pandasのオプション
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', 5000)
pd.options.display.float_format = '{:.3f}'.format
%matplotlib inline
# sns.set_style('whitegrid')
plt.style.use('fivethirtyeight')

from joblib import Parallel, delayed # よりお手軽にサクっと並列処理を実行出来るモジュール
from tqdm import tqdm, tqdm_notebook # プログレスバーを表示できる
from PIL import Image
tqdm.pandas()

# 外部モジュールを自動的にリロードする
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
CONFIG_FILE = '../configs/config.yaml'

with open(CONFIG_FILE) as file:
    yml = yaml.load(file)
MODEL_DIR_NAME = yml['SETTING']['MODEL_DIR_NAME']
FEATURE_DIR_NAME = yml['SETTING']['FEATURE_PATH']

In [84]:
def load_datasets_train(feats):
    dfs = [pd.read_pickle(FEATURE_DIR_NAME + f'{f}_train.pkl') for f in feats]
    X_train = pd.concat(dfs, axis=1)
    return X_train

def load_datasets_both(feats):
    dfs = [pd.read_pickle(FEATURE_DIR_NAME + f'{f}_train.pkl') for f in feats]
    X_train = pd.concat(dfs, axis=1)
    dfs = [pd.read_pickle(FEATURE_DIR_NAME + f'{f}_test.pkl') for f in feats]
    X_test = pd.concat(dfs, axis=1)
    return X_train, X_test

# 欠損値の確認
def missing_values_table(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return(np.transpose(tt))

In [85]:
! pwd

/Users/endotakeru/Documents/python/signate/car-run/pipeline/notebooks


In [87]:
features = [
        'acceleration',
        'car_label_encoder',
        'cylinders',
        'displacement',
        'horsepower_mis_val_median',
        'model_year',
        'mpg',
        'origin',
        'weight',
]

In [88]:
train, test = load_datasets_both(features)

In [89]:
display(train.shape, test.shape)

(199, 9)

(199, 8)

In [90]:
display(train.head(), train.tail(), train.shape)

,acceleration,car name_label_encoder,cylinders,displacement,horsepower_mis_val_median,model year,mpg,origin,weight
0,16.000,96,4,135.000,84.000,82,29.000,1,2525.000
1,14.000,304,4,89.000,71.000,79,31.900,2,1925.000
2,15.500,276,6,156.000,108.000,76,19.000,3,2930.000
3,14.500,103,4,90.000,75.000,74,28.000,1,2125.000
4,17.300,278,4,89.000,62.000,81,37.700,3,2050.000


,acceleration,car name_label_encoder,cylinders,displacement,horsepower_mis_val_median,model year,mpg,origin,weight
194,19.200,79,4,85.000,65.000,80,40.800,3,2110.000
195,12.800,195,8,302.000,139.000,78,20.200,1,3570.000
196,12.000,14,8,304.000,150.000,70,16.000,1,3433.000
197,23.700,300,4,90.000,48.000,80,43.400,2,2335.000
198,15.500,121,4,98.000,90.000,73,26.000,2,2265.000


(199, 9)

In [91]:
display(train.describe(), test.describe())

,acceleration,car name_label_encoder,cylinders,displacement,horsepower_mis_val_median,model year,mpg,origin,weight
count,199.000,199.000,199.000,199.000,199.000,199.000,199.000,199.000,199.000
mean,15.648,144.553,5.296,183.312,101.070,76.166,24.307,1.583,2883.839
std,2.702,92.657,1.645,98.400,35.237,3.803,7.798,0.799,819.767
min,8.500,0.000,3.000,71.000,46.000,70.000,9.000,1.000,1613.000
25%,14.000,61.000,4.000,98.000,75.000,73.000,18.000,1.000,2217.500
50%,15.500,140.000,4.000,140.000,90.000,76.000,24.000,1.000,2702.000
75%,17.150,227.000,6.000,250.000,112.500,80.000,30.500,2.000,3426.500
max,23.700,304.000,8.000,454.000,220.000,82.000,44.600,3.000,5140.000


,acceleration,car name_label_encoder,cylinders,displacement,horsepower_mis_val_median,model year,origin,weight
count,199.000,199.000,199.000,199.000,199.000,199.000,199.000,199.000
mean,15.488,152.548,5.613,203.540,107.482,75.854,1.563,3057.010
std,2.817,86.267,1.745,109.138,40.847,3.592,0.807,866.497
min,8.000,2.000,3.000,68.000,46.000,70.000,1.000,1795.000
25%,13.500,74.500,4.000,107.000,78.000,73.000,1.000,2255.500
50%,15.500,156.000,6.000,163.000,95.000,76.000,1.000,2945.000
75%,17.150,223.500,8.000,304.000,138.500,79.000,2.000,3672.000
max,24.800,303.000,8.000,455.000,230.000,82.000,3.000,4952.000


In [92]:
# 各データの欠損値を確認
display(
    missing_values_table(train),
    missing_values_table(test)
)

,acceleration,car name_label_encoder,cylinders,displacement,horsepower_mis_val_median,model year,mpg,origin,weight
Total,0,0,0,0,0,0,0,0,0
Percent,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Types,float64,int64,int64,float64,float64,int64,float64,int64,float64


,acceleration,car name_label_encoder,cylinders,displacement,horsepower_mis_val_median,model year,origin,weight
Total,0,0,0,0,0,0,0,0
Percent,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Types,float64,int64,int64,float64,float64,int64,int64,float64


In [93]:
# カラムのタイプを確認
display(
    train.dtypes
)

acceleration                 float64
car name_label_encoder         int64
cylinders                      int64
displacement                 float64
horsepower_mis_val_median    float64
model year                     int64
mpg                          float64
origin                         int64
weight                       float64
dtype: object

In [95]:
submission = pd.read_csv('../models/xgb_0209_1759/xgb_0209_1759_submission.csv', delimiter='\t')

In [109]:
sample_submission = pd.read_csv('../data/raw/sample_submit.csv', names=["id", "mpg"])

In [111]:
sample_submission.to_csv('sample_submit.csv', index = False)